Using SEC data to extract items of interest


In [1]:
import requests
import os
import csv
import pandas as pd
from time import sleep
CIKlookup = "https://www.sec.gov/include/ticker.txt"
CIKfile = "CIK.tsv"
secURL = "https://data.sec.gov/api/xbrl/companyfacts/CIK0000320193.json"

The code below is a function that gets the CIK value for a given ticker. 

In [2]:
def download_file(url="https://www.sec.gov/include/ticker.txt", filename="CIK.tsv", user_agent="Michelle LiuWatts admin@hercompany.com"):
    headers = {"User-Agent": user_agent}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(response.content)
        return f"File downloaded successfully as {filename}"
    else:
        return f"Failed to download file: {response.status_code}"


def get_cik_from_ticker(ticker, file_path='CIK.tsv'):
    if not os.path.exists(file_path):
        result = download_file()
        if "Failed" in result:
            return result

    with open(file_path, 'r') as file:
        reader = csv.reader(file, delimiter='\t')
        for row in reader:
            # print(row)
            if row[0] == ticker.lower():
                return row[1].zfill(10)
    return "Ticker not found."

The CIK for CRM is 0001108524


In [8]:

# Example usage
ticker = "FI"
cik = get_cik_from_ticker(ticker)
print(f"The CIK for {ticker} is {cik}")

The CIK for FI is Ticker not found.


In [3]:
spy_df = pd.read_csv("spy_holdings.csv")
# spy_df.head()
spy_df['CIK'] = spy_df['Ticker'].apply(get_cik_from_ticker)
for i, row in spy_df.iterrows():
    cik = row['CIK']
    ticker = row['Ticker']
    url = f"https://data.sec.gov/api/xbrl/companyfacts/CIK{cik}.json"
    if not os.path.exists(f"./qtrly-data/{ticker}.json"):
        result = download_file(url, f"./qtrly-data/{ticker}.json")
        sleep(0.101)
    else:
        result = f"{ticker} already downloaded"
        print(result)
    if "Failed" in result:
        print(f"{ticker} failed to download")

AAPL already downloaded
BRK.B failed to download
- failed to download
FI failed to download
GEV failed to download
KVUE failed to download
COR failed to download
GEHC failed to download
SW failed to download
VLTO failed to download
CPAY failed to download
EG failed to download
GEN failed to download
RVTY failed to download
DAY failed to download
SOLV failed to download
BF.B failed to download
